In [ ]:
import json

# 先建立 dict 格式的 account.json 檔，並取得其中的帳密資料
with open("./account.json", "r", encoding="utf-8") as file:
    strJson = file.read()
    dictJson = json.loads(strJson)

In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 regular expression 工具
import re

# 子處理程序，用來取代 os.system 的功能
import subprocess


'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
yt-dlp 下載影音的好工具
參考網址：https://github.com/yt-dlp/yt-dlp
'''
# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知


# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)


'''
自訂變數
'''
# 想爬取資料的來源網址
url = 'https://www.instagram.com/'
ig_account = 'english.ig_'

# 放置爬取的資料
listData = []

# 放置 ig 每個格子裡面的超連結
listLink = []

# 設定 set 物件，放置影片、圖片連結，協助過濾重複的元素
setTmp = set()

In [ ]:
'''
以 function 名稱，作為爬蟲流程
'''
def login():
    try:
        # 前往首頁
        driver.get(url);

        # 等待互動元素出現 (這裡用帳號文字欄位)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, 'input[name="username"]') 
            )
        )

        # 輸入帳號
        driver.find_element(
            By.CSS_SELECTOR, 
            'input[name="username"]'
        ).send_keys(dictJson['username'])

        # 輸入密碼
        driver.find_element(
            By.CSS_SELECTOR, 
            'input[name="password"]'
        ).send_keys(dictJson['password'])

        # 強制等待
        sleep(3)

        # 按下登入
        driver.find_element(
            By.CSS_SELECTOR, 
            'button[type="submit"].sqdOP.L3NKy.y3zKF'
        ).click()
        
        # 強制等待
        sleep(4)
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        driver.quit()

# 走訪頁面
def visit():
    # 前往指定連結
    driver.get(url + ig_account);

# 取得每個項目的 url
def getUrl():
    # 逐一檢視元素
    for index, a in enumerate(driver.find_elements(By.CSS_SELECTOR, 'div.Nnq7C.weEfm div.v1Nh3.kIKUG._bz0w a')):
        # 測試功能，所以只選三個連結
        #if index == 3:
        #    break
            
        print(f"取得網址: {a.get_attribute('href')}")
        
        # 放資料到 list 中
        listLink.append(a.get_attribute('href'))

# 逐個網頁連結內容進行分析
def parse():
    for aLink in listLink:
        # 清空暫存圖片、影片連結的 set
        setTmp.clear()
        
        # 走訪頁面
        driver.get(aLink)
        
        '''
        取得頁面的 id：
            full_match = re.search(regex, aLink)[0]
            group_1 = re.search(regex, aLink)[1]
        
        範例：
            matchObj[0]  : /p/CGpaFHrHV2j/
            matchObj[1]  : CGpaFHrHV2j 
        '''
        # 取得 ig 連結的 id
        pageId = re.search(r'\/p\/([a-zA-Z0-9-_]+)\/', aLink)[1]
        
        print(f"網頁 ID: {pageId}")
        
        # 強制等待
        sleep(2)
        
        '''
        判斷是否有「向右」按鈕：
            若有，則代表會有多個 li；
            若無，則代表只有一個 li

        備註：
            由於 find_elements 回傳 list 格式，
            所以可以用 len() 來取得元素長度，
            判斷元素是否存在
        '''
        # 當有右箭頭可以按的時候，代表頁面中同時有多張圖片或影片，取得多元素資訊
        while len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
            # 按下向右按鈕
            driver.find_element(By.CSS_SELECTOR, "button._6CZji").click()

            # 檢視當前現有的 li (解決滑動時，前面一部分或後面一部分 li.Ckrof 會消失的問題)
            for index, li in enumerate(driver.find_elements(By.CSS_SELECTOR, "article li.Ckrof")):
                # 判斷 li 底下是否有 img
                if len(li.find_elements(By.CSS_SELECTOR, "img.FFVAD")) > 0:
                    # 將 img 裡面的屬性值(例如 src)，視為 value 加入 set 當中
                    setTmp.add(li.find_element(By.CSS_SELECTOR, "img.FFVAD").get_attribute('src'))
                else:
                    # 若非 img，便是 video，將頁面連結加入 set 當中
                    setTmp.add(aLink)

            # 強制等待
            sleep(1)
        else:
            # 當沒有右箭頭可以按的時候，代表是獨立的單張圖片或影片
            if len( driver.find_elements(By.CSS_SELECTOR, "article img.FFVAD") ) > 0:
                # 將 img 的 src 值加入 set 當中
                setTmp.add(driver.find_element(By.CSS_SELECTOR, "article img.FFVAD").get_attribute('src'))
            else:
                # 若非 img，便是 video，將頁面連結加入 set 當中
                setTmp.add(aLink)
                

        # 整合此次網頁連結的元素資料
        listData.append({
            "id": pageId,
            "url": aLink,
            "listDL": list(setTmp)
        })
        
# 將 list 存成 json
def saveJson():
    with open("./ig.json", "w", encoding='utf-8') as fp:
        fp.write( json.dumps(listData, ensure_ascii=False, indent=4) )
    
# 關閉瀏覽器
def close():
    driver.quit()

In [ ]:
# 流程開始前，先行登入，減少被 IG 停止帳號的風險 (雖然修改密碼還是救得回來…)
login()

In [ ]:
'''主程式'''
if __name__ == '__main__':
    visit()
    getUrl()
    parse()
    saveJson()

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 下載檔案
def download():
    # 開啟 json 檔案
    with open("./ig.json", "r", encoding='utf-8') as fp:
        #取得 json 字串
        strJson = fp.read()
    
    # 建立儲存圖片、影片的資料夾
    folderPath = 'ig'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    # 將 listData 底下的 dict 找出來
    for d in listResult:
        # dictObj 底下有個 listDL 屬性，它對應的值是個 list，將 list裡面所有 url 找出來
        for index, url in enumerate(d['listDL']):
            print("=" * 50)
            print(f"正在下載連結: {url}")
            
            match = re.search(r"https:\/\/\S+\/(\S+.jpe?g)", url)
            if match != None:
                os.system(f'curl "{url}" -o ./{folderPath}/{d["id"]}_{match[1]}')
                print("下載完成")
            else:
                # 強制等待
                sleep(2)
                
                # 定義指令
                cmd = [
                    './yt-dlp.exe', 
                    url, 
                    '-f', 'b[ext=mp4]', 
                    '-o', f'./{folderPath}/%(id)s_{index}.%(ext)s', 
                    '-u', dictJson['username'],
                    '-p', dictJson['password']
                ]
                
                # 執行指令，並取得回傳結果
                result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

                # 將回傳結果進行解碼，顯示實際執行過程的文字輸出
                output = result.stdout.decode('utf-8')
                print("下載完成，訊息如下:")
                print(output)

'''另行下載'''
download()